In [1]:
from z3 import *
import math

## Utils

#### At most/least one & exactly one

In [2]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

#### Conversion from int to bin

In [3]:
def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return math.ceil(math.log2(max(sum(s), max(l))))

In [4]:
def int_to_bin(x, digits=5):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

#### Operations on binary numbers

In [5]:
### ATTEMPT by Scaio function equal
### CHANGED by Edo function leq
def equal(v, u, digits):
    return And([v[k]==u[k] for k in range(digits)])

def all_false(v, digits):
    return And([v[k]==False for k in range(digits)])

def leq(v, u, digits): #v<=u
    if digits == 1:
        return Or(v[0]==u[0], And(v[0]==False, u[0]==True))
    else:
        return Or(And(v[0]==False, u[0]==True), 
                  And(v[0]==u[0], leq(v[1:], u[1:], digits-1)))
    


In [6]:
v = [False, False]
u = [False, True]

s = Solver()
s.add(all_false(v, 2))
print( s.check())

s = Solver()
s.add(equal(v, u, 2))
print( s.check())

s = Solver()
s.add(leq(v, u, 2))
print(s.check(), 'and expected sat')

s = Solver()
s.add(leq(u, v, 2))
print(s.check(), 'and expected unsat')

sat
unsat
sat and expected sat
unsat and expected unsat


In [7]:
def sum_bin(a_bin, b_bin, d_bin, digits):
    """Encodes into a SAT formula the binary sum {a_bin + b_bin = d_bin}, each number having {digits} num of bits

    Args:
        a_bin (list[bool]): binary representation of a
        b_bin (list[bool]): binary representation of b
        d_bin (list[bool]): binary representation of d
        digits (int): number of bits of each number 

    Returns:
        (Z3-expression): formula representing SAT encoding of binary sum
    """

    #slide SAT-I
    c = [Bool(f"c_{k}") for k in range(digits+1)]
    c[0] = False
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])), 
                                   And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    return And(clauses)

In [8]:
s = Solver()
s.add(sum_bin(int_to_bin(11), int_to_bin(8), int_to_bin(19), 5))
print(s.check())

sat


In [9]:
for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 8):
            s = Solver()
            s.add(sum_bin(int_to_bin(i), int_to_bin(j), int_to_bin(k), 5))
            print(i, '+', j, '=', k, '-->', s.check())
            

1 + 1 = 1 --> unsat
1 + 1 = 2 --> sat
1 + 1 = 3 --> unsat
1 + 1 = 4 --> unsat
1 + 1 = 5 --> unsat
1 + 1 = 6 --> unsat
1 + 1 = 7 --> unsat
1 + 2 = 1 --> unsat
1 + 2 = 2 --> unsat
1 + 2 = 3 --> sat
1 + 2 = 4 --> unsat
1 + 2 = 5 --> unsat
1 + 2 = 6 --> unsat
1 + 2 = 7 --> unsat
1 + 3 = 1 --> unsat
1 + 3 = 2 --> unsat
1 + 3 = 3 --> unsat
1 + 3 = 4 --> sat
1 + 3 = 5 --> unsat
1 + 3 = 6 --> unsat
1 + 3 = 7 --> unsat
2 + 1 = 1 --> unsat
2 + 1 = 2 --> unsat
2 + 1 = 3 --> sat
2 + 1 = 4 --> unsat
2 + 1 = 5 --> unsat
2 + 1 = 6 --> unsat
2 + 1 = 7 --> unsat
2 + 2 = 1 --> unsat
2 + 2 = 2 --> unsat
2 + 2 = 3 --> unsat
2 + 2 = 4 --> sat
2 + 2 = 5 --> unsat
2 + 2 = 6 --> unsat
2 + 2 = 7 --> unsat
2 + 3 = 1 --> unsat
2 + 3 = 2 --> unsat
2 + 3 = 3 --> unsat
2 + 3 = 4 --> unsat
2 + 3 = 5 --> sat
2 + 3 = 6 --> unsat
2 + 3 = 7 --> unsat
3 + 1 = 1 --> unsat
3 + 1 = 2 --> unsat
3 + 1 = 3 --> unsat
3 + 1 = 4 --> sat
3 + 1 = 5 --> unsat
3 + 1 = 6 --> unsat
3 + 1 = 7 --> unsat
3 + 2 = 1 --> unsat
3 + 2 = 2 --> 

In [10]:
def LinearInteger(c_i, s, l_i, i, n, digits):
    """Encodes into a SAT formula the linear constraint {sum_over_j(s_i[j] | c_i[j] == True) <= l_i}

    Args:
        c_i (list[bool]): a[i,:] in model 1
        s (list[list[bool]]): list of sizes of objects, each one represented as list[bool] i.e. binary number
        l_i (int): load of courier i        #TODO: change to l_i_bin and convert in calling function?
        i (int): index of courier (doing the job of name)
        n (int): number of objects
        digits (int): number of digits required to represent l_i and numbers in s

    Returns:
        formula (Z3-expression): And of clauses representing SAT encoding of Linear Integer constraint
    """
    # matrix containing temporary results of sum_bin
    d = [[Bool(f"d_{j}_{k}_{i}") for k in range(digits)] for j in range(n)]

    clauses = []
    # TODO: aggiungere "if c_i[j] = 1, fai la roba sotto, altrienti d_j == d_j-1"
    for j in range(n):
        clauses.append(sum_bin(d[j-1], s[j], d[j], digits)) # temporary result d_j-1 + s_j = (imposed) d_j



    # ### ATTEMPT by Scaio for the if condition above

    # # with function equal  
    # # row 0
    # clauses.append( Or(And(c_i[0], equal(int_to_bin(s[0]), d[0, :], digits)),       # change with Implies()
    #                    And(Not(c_i[0]), all_false(d[0, :], digits))) )
    # # row j>1
    # for j in range(1,n):
    #     clauses.append( Or(And(c_i[j],sum_bin(d[j-1, :], s[j], d[j, :])),                # c_j True and SUM True  
    #                        And(Not(c_i[j]), equal(d[j, :], d[j-1, :], digits))) )    # c_j False and d_j==d_j-1

    # # TODO: aggiungere a clause che d_n <= l_i
    # # I suppose to enter l_i as integer (is it the best idea to optimize the program?)
    # l_i_bin = int_to_bin(l_i, digits)
    # clauses.append(leq(d[j-1, :], l_i_bin, digits)) 

    # ### END


    ### Taken from Scaio attempt above and CHANGED by Edo
    # row 0
    clauses.append( And(Implies(c_i[0], equal(d[0], s[0], digits)),      # If c_i[0] == 1 then d_0 == s_0
                        Implies(Not(c_i[0]), all_false(d[0], digits))))              # elif c_i[0] == 0 then d_0 == [0..0]
    
    # row j>1
    for j in range(1,n):
        clauses.append( And(Implies(c_i[j], sum_bin(d[j-1], s[j], d[j], digits)),               # If c_j == 1 then d_j == d_j-1 + s_j 
                            Implies(Not(c_i[j]), equal(d[j], d[j-1], digits))))     # elif c_j == 0 then d_j == d_j-1

    # TODO: aggiungere a clause che d_n <= l_i
    # I suppose to enter l_i as integer (is it the best idea to optimize the program?)
    l_i_bin = int_to_bin(l_i, digits)
    clauses.append(leq(d[n-1], l_i_bin, digits)) 
    # TODO: does it make any sense to check d_j <= l_i_bin forall j or better to just do it for d_n?
    # TODO: have to check for overflows in num. digits?

    ### END

    formula = And(clauses)
    return formula



## Model 1

In [13]:
def multiple_couriers_planning_1(m, n, l, s, D):
    """_summary_

    Args:
        m (_type_): _description_
        n (_type_): _description_
        l (_type_): _description_
        s (_type_): _description_
        D (_type_): _description_
    """
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{k}_{j}_{i}") for k in range(n)] for j in range(n)] for i in range(m)]  
    # r_kji = 1 indicates that courier i delivers object j as its k-th delivery
    # O(m * n^2) vars

    solver = Solver()


    ## CONSTRAINTS
    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Constraint 2: every courier can't exceed its load capacity
    digits = find_max_digits(s, l)
    s_bin = [int_to_bin(s_j, digits) for s_j in s]
    for i in range(m):
        solver.add(LinearInteger(a[i], s_bin, l[i], i, n, digits))

    # Interesting question: see discussion forum (distance func. property)
    # -> answer: the distance is quasimetric => we can add implicit constraint "every courier has at least 1 package to deliver"
    # TODO: check if it actually speeds up execution

    print(solver.check())


### Testing Model 1

In [14]:
m = 3
n = 7
l = [15, 10, 7]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]

multiple_couriers_planning_1(m, n, l, s, D)

unsat
